<img src="assets/logo.png" width="300px">

# Supplementary 2 - Cross-Validation and Model Stability Assessment
**BoutScout: A Deep Learning Framework for Automatic Detection of Incubation Events in Avian Nests Using Temperature Time Series**

Author: [Jorge Lizarazo](https://www.researchgate.net/profile/Jorge-Lizarazo-Borrero?ev=hdr_xprf)


This supplementary notebook presents the cross-validation procedure used to assess the generalization and robustness of the BoutScout model. Using a 5-fold StratifiedKFold approach, we evaluated model performance across multiple splits while ensuring balanced class distributions per fold. For each fold, the BiLSTM model was reinitialized and trained from scratch for 50 epochs, using identical architecture and training settings.

In addition to fold-specific plots, this notebook includes mean loss curves, average precision-recall curves with standard deviation bands, and normalized confusion matrices across folds to illustrate performance consistency.

**Year:** 2025


In [40]:
import numpy as np
import glob
import os
import pandas as pd
import json
import ast
import torch.nn as nn
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch
import torch.optim as optim
from scipy.interpolate import interp1d
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize

In [42]:
#print(torch.__version__)
#print(torch.cuda.is_available())
#print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

In [43]:
# Kargar arrays
X_array = np.load("X_array_cleaned.npy", allow_pickle=True)
y_array = np.load("y_array_cleaned.npy", allow_pickle=True)

# Kargar clases del LabelEncoder
le = LabelEncoder()
le.classes_ = np.load("label_classes.npy", allow_pickle=True)

# Definir Dataset
class NestEventDataset(Dataset):
    def __init__(self, X_array, y_array):
        self.X_array = X_array
        self.y_array = y_array

    def __len__(self):
        return len(self.X_array)

    def __getitem__(self, idx):
        X = self.X_array[idx].astype(np.float32)
        y = self.y_array[idx].astype(np.int64)
        return torch.tensor(X), torch.tensor(y)

# Crear el dataset
dataset = NestEventDataset(X_array, y_array)

In [44]:

idx = random.randint(0, len(dataset) - 1)
X_sample, y_sample = dataset[idx]
X_np = X_sample.numpy()
y_np = y_sample.numpy()

In [45]:
print(f"Numero total de dias (entradas): {len(y_array)}")

Número total de días (entradas): 2232


In [46]:
idx = random.randint(0, len(X_array) - 1)
X_sample = X_array[idx].astype(np.float32)
y_sample = y_array[idx].astype(np.int64)

X_np = X_sample
y_np = y_sample

In [47]:
y_np

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [48]:
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTMModel, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        return out

In [49]:
input_size = X_array[0].shape[1]  # número de features por minuto (ej: temperatura, ambiente, etc.)
hidden_size = 64
num_layers = 2
num_classes = 3  # ['Error', 'Nocturnal', 'Off', 'On'], aunque eliminaste "Error", mantenemos por consistencia

model = BiLSTMModel(input_size, hidden_size, num_layers, num_classes)

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [52]:
from sklearn.model_selection import StratifiedKFold

import torch.nn as nn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, average_precision_score
from sklearn.metrics import classification_report
import os

We set up directories to save confusion matrices, precision-recall curves, and loss plots.

In [53]:
os.makedirs("figures/confusion_matrices", exist_ok=True)
os.makedirs("figures/precision_recall", exist_ok=True)
os.makedirs("figures/loss_curves", exist_ok=True)

Definning dominant class per day for stratification, create result folders, and set up 5-fold StratifiedKFold.

In [54]:
# Etiqueta por día dominante
# Crear carpetas de salida si no existen
os.makedirs("resultados_folds", exist_ok=True)
os.makedirs("modelos_folds", exist_ok=True)

mejor_f1 = 0
mejor_fold = -1
y_mode = [np.bincount(y).argmax() for y in y_array]


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_scores = []
ruta_mejor_modelo = "modelos/modelo_bilstm_cv_mejor_cross.pth"

## 5-Fold Cross-Validation Training Loop
We perform 5-fold cross-validation using a StratifiedKFold split based on each day's dominant label.
For each fold:
- The BiLSTM model is reinitialized and trained for 50 epochs.
- Training and validation losses are computed.
- Predictions are collected to compute F1-score, classification reports, and confusion matrices.
- Training losses and predictions are saved for later analysis.
- The model with the best macro F1-score across folds is saved for deployment.

In [57]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X_array, y_mode), 1):
    print(f"\n=== Fold {fold}/5 ===")

    X_train = [X_array[i] for i in train_idx]
    y_train = [y_array[i] for i in train_idx]
    X_val = [X_array[i] for i in val_idx]
    y_val = [y_array[i] for i in val_idx]

    train_ds = NestEventDataset(X_train, y_train)
    val_ds = NestEventDataset(X_val, y_val)

    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

    model = BiLSTMModel(input_size=X_array[0].shape[1], hidden_size=64, num_layers=2, num_classes=3).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    train_losses = []

    for epoch in range(50):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs.view(-1, outputs.shape[-1]), y_batch.view(-1))
            loss.backward()
            train_losses.append(loss.item())
            optimizer.step()

    # Evaluación
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=2).view(-1).cpu().numpy()
            labels = y_batch.view(-1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)

        np.save(f"resultados_folds/fold_{fold}_train_losses.npy", np.array(train_losses))
        np.save(f"resultados_folds/fold_{fold}_labels.npy", np.array(all_labels))
        np.save(f"resultados_folds/fold_{fold}_preds.npy", np.array(all_preds))

    report = classification_report(all_labels, all_preds, target_names=['Nocturnal', 'Off', 'On'], zero_division=0, output_dict=True)
    f1_macro = report['macro avg']['f1-score']
    fold_scores.append(f1_macro)

    print(f"F1 macro (fold {fold}): {f1_macro:.4f}")

    # === Guardar resultados y modelo de este fold ===
    with open(f"resultados_folds/fold_{fold}_reporte.json", "w") as f:
        json.dump(report, f, indent=4)

    torch.save(model.state_dict(), f"modelos_folds/modelo_fold_{fold}.pth")

    if f1_macro > mejor_f1:
        mejor_f1 = f1_macro
        mejor_fold = fold
        torch.save(model.state_dict(), ruta_mejor_modelo)
        print(f"✅ Guardado mejor modelo (F1: {mejor_f1:.4f}) en fold {mejor_fold}")

print(f"\n=== Promedio F1 macro en validación cruzada: {np.mean(fold_scores):.4f} ===")
print(f"🏆 Mejor modelo guardado fue del fold {mejor_fold} con F1 macro = {mejor_f1:.4f}")


=== Fold 1/5 ===
F1 macro (fold 1): 0.9089

=== Fold 2/5 ===
F1 macro (fold 2): 0.9235

=== Fold 3/5 ===
F1 macro (fold 3): 0.9433

=== Fold 4/5 ===
F1 macro (fold 4): 0.9577
✅ Guardado mejor modelo (F1: 0.9577) en fold 4

=== Fold 5/5 ===
F1 macro (fold 5): 0.9520

=== Promedio F1 macro en validación cruzada: 0.9419 ===
🏆 Mejor modelo guardado fue del fold 4 con F1 macro = 0.9577


## Per-fold visualizations
We generate and save key diagnostic plots for each fold:
- **Training loss curves** to monitor convergence.
- **Confusion matrices** to assess class-specific performance.
- **Precision–recall curves** with average precision (AP) per class.

In [58]:
folds = [1, 2, 3, 4, 5]
class_names = ['Nocturnal', 'Off', 'On']
os.makedirs("figures/confusion_matrices", exist_ok=True)
os.makedirs("figures/precision_recall", exist_ok=True)
os.makedirs("figures/loss_curves", exist_ok=True)

for fold in folds:
    losses = np.load(f"resultados_folds/fold_{fold}_train_losses.npy")
    labels = np.load(f"resultados_folds/fold_{fold}_labels.npy")
    preds = np.load(f"resultados_folds/fold_{fold}_preds.npy")

    # 1. Loss Curve
    plt.figure()
    plt.plot(losses, label='Training Loss', color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training Loss - Fold {fold}')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"figures/loss_curves/fold_{fold}_loss.png")
    plt.close()

    # 2. Confusion Matrix
    cm = confusion_matrix(labels, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(ax=ax, cmap='Blues', values_format='d')
    plt.title(f'Confusion Matrix - Fold {fold}')
    plt.savefig(f"figures/confusion_matrices/fold_{fold}_confusion_matrix.png")
    plt.close()

    # 3. Precision-Recall Curve
    y_true_bin = label_binarize(labels, classes=[0, 1, 2])
    y_score_bin = label_binarize(preds, classes=[0, 1, 2])
    plt.figure(figsize=(7, 5))
    for i, class_name in enumerate(class_names):
        precision, recall, _ = precision_recall_curve(y_true_bin[:, i], y_score_bin[:, i])
        ap = average_precision_score(y_true_bin[:, i], y_score_bin[:, i])
        plt.plot(recall, precision, lw=2, label=f'{class_name} (AP = {ap:.2f})')

    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f'Precision-Recall Curve - Fold {fold}')
    plt.legend(loc="best")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"figures/precision_recall/fold_{fold}_pr_curve.png")
    plt.close()

NameError: name 'label_binarize' is not defined

### Mean loss curve across folds
To evaluate the overall training behavior of the model, we aggregate the training loss curves from all five cross-validation folds. We compute the mean loss per epoch and the standard deviation across folds to assess convergence consistency. The resulting plot shows the average training trajectory with a shaded area representing ±1 standard deviation, helping visualize how stable the learning process is across splits.

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
folds = [1, 2, 3, 4, 5]
all_losses = []

# Recolectar todas las curvas de pérdida
for fold in folds:
    losses = np.load(f"resultados_folds/fold_{fold}_train_losses.npy")
    all_losses.append(losses)

# Convertir a array para calcular promedio y desviación estándar
all_losses = np.array(all_losses)
mean_loss = all_losses.mean(axis=0)
std_loss = all_losses.std(axis=0)

# Plot
plt.figure(figsize=(8, 5))
plt.plot(mean_loss, label='Mean Loss', color='#707070', linewidth=2)
plt.fill_between(range(len(mean_loss)), mean_loss - std_loss, mean_loss + std_loss,
                 color='#C7C7C7', alpha=0.3, label='±1 Std. Dev')

plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.title('Training Loss per Epoch\nMean and Std. Deviation across Folds')
plt.grid(False)
plt.legend()
plt.tight_layout()
# plt.savefig("figures/loss_curves/mean_loss_curve_only.png")
plt.show()


### Mean precision–recall curves by class
We compute average precision–recall (PR) curves across folds for each behavioral class (`Nocturnal`, `Off`, and `On`).
To ensure comparability, precision values are interpolated over a fixed recall grid from 0 to 1.
We then calculate the mean and standard deviation of precision across folds, and plot shaded bands to represent variability.
This visualization allows us to evaluate the model’s ability to distinguish each class consistently across different data splits.

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
folds = [1, 2, 3, 4, 5]
class_names = ['Nocturnal', 'Off', 'On']

# Colores personalizados por clase
custom_colors = {
    'On': '#E28342',
    'Off': '#535AA6',
    'Nocturnal': '#333E48'
}

# Puntos comunes de recall (de 0 a 1)
recall_grid = np.linspace(0, 1, 100)

plt.figure(figsize=(8, 6))

for class_idx, class_name in enumerate(class_names):
    interpolated_precisions = []

    for fold in folds:
        labels = np.load(f"resultados_folds/fold_{fold}_labels.npy")
        preds = np.load(f"resultados_folds/fold_{fold}_preds.npy")

        y_true_bin = label_binarize(labels, classes=[0, 1, 2])[:, class_idx]
        y_pred_bin = label_binarize(preds, classes=[0, 1, 2])[:, class_idx]

        precision, recall, _ = precision_recall_curve(y_true_bin, y_pred_bin)

        # Interpolar precisión en los mismos valores de recall
        interp = interp1d(recall, precision, kind='linear', bounds_error=False,
                          fill_value=(precision[0], precision[-1]))
        interp_prec = interp(recall_grid)
        interpolated_precisions.append(interp_prec)

    interpolated_precisions = np.array(interpolated_precisions)
    mean_precision = interpolated_precisions.mean(axis=0)
    std_precision = interpolated_precisions.std(axis=0)

    color = custom_colors[class_name]

    # Plot media + área de desviación estándar
    plt.plot(recall_grid, mean_precision, label=class_name, linewidth=2, color=color)
    plt.fill_between(recall_grid, mean_precision - std_precision, mean_precision + std_precision,
                     alpha=0.2, color=color)

plt.xlabel('Recall')
plt.ylabel('Precision')
#plt.title('Mean Precision-Recall Curves with Std. Deviation')
plt.legend(title="Class")
plt.grid(False)
plt.tight_layout()
#plt.savefig("figures/precision_recall/mean_PR_curves_all_classes.png")
plt.show()

### Normalized confusion matrices across folds
We visualize the class-wise prediction accuracy for each fold using confusion matrices normalized by row (i.e., per true class).
Each matrix shows the proportion of correctly and incorrectly classified time steps per behavioral classes.
Grayscale shading is used to highlight prediction strength, with values displayed as percentages.
This view allows for quick comparison of classification performance consistency across folds and highlights typical confusion patterns between similar behaviors.

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
folds = [1, 2, 3, 4, 5]
class_names = ['Nocturnal', 'Off', 'On']


# Crear paleta de grises personalizada del claro al oscuro
custom_greys = ListedColormap(["#EAEAEA", "#C7C7C7", "#A0A0A0", "#707070"])

fig, axes = plt.subplots(1, 5, figsize=(14, 4))

for i, fold in enumerate(folds):
    labels = np.load(f"resultados_folds/fold_{fold}_labels.npy")
    preds = np.load(f"resultados_folds/fold_{fold}_preds.npy")

    # Normalizar por fila para obtener porcentajes
    cm = confusion_matrix(labels, preds, normalize='true') * 100
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

    disp.plot(ax=axes[i], cmap=custom_greys, colorbar=False)  # ✅ Colores grises
    axes[i].set_title(f'Fold {fold}', fontsize=16)
    axes[i].set_xlabel('Predicted', fontsize=13)
    axes[i].set_ylabel('True', fontsize=13)
    axes[i].grid(False)  # ✅ Eliminar cuadrícula

    for text in axes[i].texts:
        val = float(text.get_text())
        text.set_text(f'{val:.1f}%')
        text.set_fontsize(10)

#plt.suptitle('Normalized Confusion Matrices (% per row)', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
#plt.savefig("figures/Normalized_Confusion_Matrice.png", dpi=300)
plt.show()

Please go to the next, [Supplementary 3](https://github.com/jorgelizarazo94/BoutScout_Model/blob/05fdad347af45f8804484a6c9b0b3592751f4a83/supplementary_3.ipynb)